In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Define class labels
class_labels = ['area', 'heatmap', 'horizontal_bar', 'horizontal_interval', 'line', 'manhattan',
               'map', 'pie', 'scatter', 'scatter-line', 'surface', 'venn', 'vertical_bar',
               'vertical_box', 'vertical_interval']
num_classes = len(class_labels)

In [ ]:
# Adjust paths to your train folder and class labels
train_path = "/kaggle/input/icpr-2022/ICPR 2022/ICPR2022_CHARTINFO_UB_PMC_TRAIN_v1.0/images"

# Data augmentation settings
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical',
    classes = class_labels,
    shuffle = False  
)

# Split training data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_datagen.filenames,
    train_datagen.classes,
    test_size=0.3,  # 30% for validation
    stratify=train_datagen.classes  # Maintain class balance
)

test_generator = (val_data, val_labels)

In [ ]:
from keras.model import Model
from keras.layers import Input,ConvLSTM2D,Dense,Dropout,GlobalAveragePooling2D,BatchNormalization,Multiply

In [ ]:
def get_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Gated Attention Block
    def GatedAttentionBlock(spatial_inputs, num_features, red_coeff=8):
        x = ConvLSTM2D(filters=(num_features//red_coeff), kernal_size=(3,3), padding="same", return_sequences=True)(spatial_inputs)
        x = ConvLSTM2D(filters=num_features, kernal_size=(3,3), padding="same", return_sequences=True)(x)
        AttMaps = Multiply()([spatial_inputs, x])
        return AttMaps
    
    # MLP function
    def mlp(x, embedding_dim, mlp_dim=256, drop_rate=0.2):
        x = Dense(mlp_dim, activation='relu')(x)
        x = Dropout(drop_rate)(x)
        x = Dense(embedding_dim, activation='sigmoid')(x)
        x = Dropout(drop_rate)(x)
        return x
    
    x = BatchNormalization()(inputs)
    x = GatedAttentionBlock(x, x.get_Shape().as_list()[-1])
    
    x = GlobalAveragePooling2D()(x)
    x = mlp(x, x.get_shape().as_list()[-1], mlp_dim=128, drop_rate=0.2)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


input_shape = (img_width, img_height, 3)

# Create the model
model = get_model(input_shape, num_classes)

#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    epochs = 10,
    validation_Data=test_generator
)

In [ ]:
# Evaluate the model on the test set
scores = model.evaluate(test_generator)
print("Test Loss:",scores[0])
print("Test Accuracy", scores[1])